In [3]:
import os
# import lxml
from lxml import etree
import pandas as pd
import numpy as np

In [4]:
import sys

In [44]:
import os

import gensim
import pymorphy2
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC

In [45]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Convolution1D, GlobalMaxPooling1D
from keras.layers.recurrent import LSTM

Using TensorFlow backend.


In [6]:
gensim.__version__

'1.0.1'

In [17]:
cl = ['эссе','интервью','объявление']

In [18]:
f = open('out.csv', 'r')

file_hash = {}
for line in f:
    attrs = line.split(';')
    path = attrs[0]
    class_name = attrs[8]
    if ' ' in class_name:
        class_name = class_name.split()[0]
    if '|' in class_name:
        class_name = class_name.split('|')[0]
    filename = os.path.basename(path)
    if class_name in cl:
        file_hash[filename] = class_name


In [10]:
morph = pymorphy2.MorphAnalyzer()
def normalize_and_add_pos(word):
    p = morph.parse(word)[0]
    return '{}_{}'.format(
        p.normal_form,
        p.tag.POS
    )

In [11]:
def read_txt(path):
    for root, dirs, files in os.walk(path):
        for filename in files:
            open_name = os.path.join(root, filename)
            if filename in file_hash:
                class_text = file_hash[filename]
                tree = etree.parse(open_name)
                yield tree, class_text

In [19]:
sent = []
for tree, class_text in read_txt('/Users/Valeriya/Desktop/CoursePaper/corpus'):
    all_p = tree.xpath('//p')
    joined = '\n'.join([p.text for p in all_p if p.text is not None])
    sent.append((joined, class_text))

In [20]:
len(sent)

445

In [21]:
only_texts = [text[0] for text in sent]
classes = [text[1] for text in sent]

In [22]:
set(classes)

{'интервью', 'объявление', 'эссе'}

In [26]:
m = 'ruwikiruscorpora_0_300_20.bin.gz'
model = gensim.models.KeyedVectors.load_word2vec_format(m, binary=True)

In [27]:
texts_norm = []
for text in sent:
    text_vectors = []
    words = text[0].split()
    for word in words:
        norm_word = normalize_and_add_pos(word)
        if norm_word in model:
            text_vectors.append(model[norm_word])
    mean_vector = sum(text_vectors) / len(text_vectors)
    texts_norm.append(mean_vector)

In [28]:
data = pd.DataFrame(texts_norm)

In [29]:
data['class'] = classes

In [30]:
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:,:300], data['class'], test_size=0.3)

In [31]:
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [37]:
svc = SVC()
svc.fit(X_train, y_train)
y_pred_svc = svc.predict(X_test)

In [32]:
accuracy_score(y_test, y_pred)

0.91044776119402981

In [38]:
accuracy_score(y_test, y_pred_svc)

0.80597014925373134

In [39]:
regr = LogisticRegression()
regr.fit(X_train, y_train)
y_pred_regr = regr.predict(X_test)

In [40]:
accuracy_score(y_test, y_pred_regr)

0.80597014925373134

In [41]:
y_pred = OneVsRestClassifier(LinearSVC(random_state=0)).fit(X_train, y_train).predict(X_test)

In [42]:
accuracy_score(y_test, y_pred)

0.87313432835820892

Нейросеть

In [46]:
model = Sequential()
model.add(LSTM(500, return_sequences=True, input_shape=(300,1)))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=False))
model.add(Dropout(0.3))
model.add(Dense(2))
model.add(Activation('softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'], show_accuracy = True, verbose = 2, validation_data = (X_train, y_train))

In [48]:
model.fit(X_train, y_train, batch_size=100, nb_epoch=10)

ValueError: Error when checking model input: expected lstm_input_1 to have 3 dimensions, but got array with shape (311, 300)